In [16]:
import pandas as pd 
from IPython.display import display, Markdown
from scipy.stats import chi2_contingency
import numpy as np

In [2]:
# Load data files
df_survey = pd.read_excel('data/private_dataL.xlsx', sheet_name='Sheet 1')
df_results = pd.read_excel('data/public_data_resultsL.xlsx', sheet_name='Sheet 1')

## Part1 - Data extracting and preparation 

### Survey data counts extracting

In [10]:
import pandas as pd

survey_counts = pd.crosstab(
    df_survey['evote'].map({0: 'Offline votes', 1: 'E-votes'}),
    df_survey['party']
)

# Rename and reset index
survey_counts.index.name = 'party'
survey_counts = survey_counts.reset_index()

# Remove column group name
survey_counts.columns.name = None

# Reorder columns
survey_counts = survey_counts[["party", "Red", "Green", "Invalid vote"]]

display(Markdown("### Survey results counts"))

# Add total column and total row
survey_counts["Total"] = survey_counts[["Red", "Green", "Invalid vote"]].sum(axis=1)
total_row = pd.DataFrame([{
    "party": "Total",
    **survey_counts[["Red", "Green", "Invalid vote", "Total"]].sum().to_dict()
}])
survey_counts = pd.concat([survey_counts, total_row], ignore_index=True)

survey_counts

### Survey results counts

,party,Red,Green,Invalid vote,Total
0,E-votes,22,41,0,63
1,Offline votes,48,86,3,137
2,Total,70,127,3,200


### Election results data counts extracting

In [4]:
# Drop 'Total' column if it exists
df_results = df_results.drop(columns=["Total"], errors="ignore")

# Map Polling stations to Offline votes and sum their votes for each party
offline_mask = df_results["party"].str.startswith("Polling station", na=False)
offline_sum = df_results.loc[offline_mask, ["Red", "Green", "Invalid ballots"]].sum(numeric_only=True)

# Offline Votes
offline_row = pd.DataFrame([{
    "party": "Offline votes",
    **offline_sum.to_dict()
}])

# E-votes only
eVotes = df_results[df_results["party"] == "E-votes"]

# Combine
results_counts = pd.concat([eVotes, offline_row], ignore_index=True)

# Reorder columns
results_counts = results_counts[["party", "Red", "Green", "Invalid ballots"]]

# renamme "Invalid ballots" to "Invalid vote" for consistency
results_counts = results_counts.rename(columns={"Invalid ballots": "Invalid vote"})

# Add total column and total row
results_counts["Total"] = results_counts[["Red", "Green", "Invalid vote"]].sum(axis=1)
total_row = pd.DataFrame([{
    "party": "Total",
    **results_counts[["Red", "Green", "Invalid vote", "Total"]].sum().to_dict()
}])
results_counts = pd.concat([results_counts, total_row], ignore_index=True)

display(Markdown("### Election results counts"))
results_counts

### Election results counts

,party,Red,Green,Invalid vote,Total
0,E-votes,130,206,1,337
1,Offline votes,278,406,18,702
2,Total,408,612,19,1039


## Part2 - Calculating shares from counts and Difference in Elections

### Survey shares between parties

In [5]:
survey = survey_counts.query("party != 'Total'").copy()

for c in ["Red", "Green", "Invalid vote"]:
    survey[c] = survey[c] / survey["Total"] * 100

survey_shares = survey[["party", "Red", "Green", "Invalid vote"]].round(1)

display(Markdown("### Survey results shares"))
survey_shares

### Survey results shares

,party,Red,Green,Invalid vote
0,E-votes,34.9,65.1,0.0
1,Offline votes,35.0,62.8,2.2


### Election shares between paries

In [6]:
results = results_counts.query("party != 'Total'").copy()

for c in ["Red", "Green", "Invalid vote"]:
    results[c] = results[c] / results["Total"] * 100

results_shares = results[["party", "Red", "Green", "Invalid vote"]].round(1)

display(Markdown("### Elections results shares"))
results_shares

### Elections results shares

,party,Red,Green,Invalid vote
0,E-votes,38.6,61.1,0.3
1,Offline votes,39.6,57.8,2.6


###  Change (Election − Survey) 

In [11]:
change = results_shares.copy()
for c in ["Red", "Green", "Invalid vote"]:
    diff = (results_shares[c] - survey_shares[c]).round(1)
    change[c] = diff.apply(lambda x: f"+{x}" if x > 0 else f"{x}")

change["party"] = ["E-votes", "Offline votes"]

display(Markdown("### Change in shares (results - survey)"))
change

### Change in shares (results - survey)

,party,Red,Green,Invalid vote
0,E-votes,+3.7,-4.0,+0.3
1,Offline votes,+4.6,-5.0,+0.4


### Part3 - Statistical test -> Hypothesis testing using Chi-squared test

We test whether the distribution of votes across parties** differs between  
the survey and the actual election results, separately for each voting channel.

#### For each voting channel (E-votes / Offline votes):

**Null hypothesis (H₀):**  
The distribution of votes across parties *(Red, Green, Invalid)* is the same  
in the survey and in the election results.

**Alternative hypothesis (H₁):**  
The distribution of votes across parties *(Red, Green, Invalid)* differs  
between the survey and the election results.

---

**Significance level:** α = 0.05  
**Decision rule:**  
- If *p* < 0.05 → reject H₀ (significant difference)  
- If *p* ≥ 0.05 → fail to reject H₀ (no significant difference)

In [ ]:
# Choosing just parties (excluding Total column)
value_cols = ["Red", "Green", "Invalid vote"]

# Chi-square function for one channel
def test_channel(channel):
    survey_row = survey_counts.loc[survey_counts["party"] == channel, value_cols].values[0]
    result_row = results_counts.loc[results_counts["party"] == channel, value_cols].values[0]
    table = np.array([survey_row, result_row])
    
    chi2, p, dof, expected = chi2_contingency(table, correction=False)
    return {
        "channel": channel,
        "chi2": round(chi2, 3),
        "df": dof,
        "p-value": round(p, 5)
    }

# perform tests for both (evotes and offline votes) channels
tests = [test_channel("E-votes"), test_channel("Offline votes")]

pd.DataFrame(tests)

,channel,chi2,df,p-value
0,E-votes,0.508,2,0.77578
1,Offline votes,1.156,2,0.56111
